In [ ]:
import requests                # Include HTTP Requests module
from bs4 import BeautifulSoup  # Include BS web scraping module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_

In [ ]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)

stmt = select([games.c.id]).where(not_(games.columns.id.like('error%')))
#for i in games.c:
#    print(i.name,i.type)
#stmt = select([games]).where(games.columns.id == gameid)
#results = conn.execute(stmt).fetchall()
#if not results:
#    conn.execute(games.insert(), id=gameid)
#stmt = select([games]).where(games.columns.id == gameid)
results = conn.execute(stmt).fetchall()
results = [value for value, in results]
#results


In [ ]:
def createPBP(playsRaw,team):
    df = pd.DataFrame()
    for i in playsRaw:
      t = i.find('td',attrs={'class':'time'}).get_text().strip()
      x = i.find('span',attrs={'class':'text'}).get_text().strip()
      sc = i.find('td', attrs={'class':'score'}).get_text().strip()
      scA = 0
      scH = 0
      if sc:
        scH = sc.split('-')[0]
        scA = sc.split('-')[1]
      df = pd.concat([df,pd.DataFrame([[t,x,scH,scA]],columns=['gameTime','action','scoreHome','scoreAway'])])
    df['team'] = team
    return df
    
def scrapeGame(gameID):
    url = "https://d3hoops.prestosports.com/seasons/men/2019-20/boxscores/" + gameID + ".xml?view=plays"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    #soup.prettify()
    home = soup.find_all('tr', attrs = {'class':'row home'})
    away = soup.find_all('tr', attrs = {'class':'row visitor'})
    #gameid = url.replace('//','/').split('/')[6].split('?')[0].replace('.xml','')

    df = pd.DataFrame()
    
    df = pd.concat([df,createPBP(home,'Home')])
    df = pd.concat([df,createPBP(away,'Away')])

    df['player'] = df.action.str.extract('([^a-z]{2,})')
    df['action'] = df.action.str.replace('([^a-z0-9\._]{2,})','')#.str.split()

    df = df.set_index('gameTime')
    df = df.sort_index(ascending=False)
    df['time'] = df.index
    try:
        df['duration'] = pd.to_datetime(df['time'].astype(str)).diff(-1).dt.total_seconds().div(60)
    except:
        df['duration'] = 0
    df['id'] = 0
    df['gameID'] = gameID

    return df[['id','gameID','time','action','scoreHome','scoreAway','team','duration']]

In [ ]:
processed = []
errored = []
n = 1
z = len(results)

allGames = pd.DataFrame()

for i in results:
    try:
        allGames = pd.concat([allGames,scrapeGame(i)])
        processed.append(i)
    except:
        errored.append(i)
    print(str(n) + " / " + str(z) + " records processed.")
    n += 1

In [ ]:
for r in results:
    conn.execute(events.delete(events.c.gameID == r))

allGames.to_sql('events', con=engine, if_exists='append', index=False)

In [ ]:
conn.close()